In [1]:
import sys
import pandas as pd
import datetime

#### Importing Awake Data using the Google Sheets API


In [7]:
# Importing awake data gathering script
from gather_awake_data import main

raw_awake_df = main()

# Creating a copy to avoid overusing the API
awake_df = raw_awake_df.copy()

# Splitting the 'Date' column into 'DayOfWeek' and 'Date' columns
awake_df[['DayOfWeek', 'Date']] = awake_df['Date'].str.split(', ', expand=True)

# Convert 'Date' column to datetime format with the specified format
awake_df['Date'] = pd.to_datetime(awake_df['Date'], format='%m/%d/%y')

# Read the 'database.csv' file and pull the most recent date
database_df = pd.read_csv('database.csv')
# Convert the 'Date' column to datetime format
database_df['Date'] = pd.to_datetime(database_df['Date'])
# Get the most recent date from the 'Date' column
latest_date = database_df['Date'].max()
if pd.isnull(latest_date):
    latest_date = pd.to_datetime('2024-01-01')

# Selecting new awake data
new_awake_df = awake_df[awake_df['Date'] > latest_date]
new_awake_df

,Date,Sleep Quality,Meal Quality,Energy Level,Weather Qual.,Socialization,Job Hours,Work Prod.,Chore Prod.,Work Out,...,Shower,Caffeine,Partner Hours,Gaming Hours,TV Hours,App Hours,Stress Level,Day Rating,EOD Emotion,DayOfWeek
35,2024-01-02,4,3,3,2,4,0,3,1,0,...,0,,14,0,3,1,3,4,unsure,Tuesday
36,2024-01-03,3,3,3,4,3,0,4,0,0,...,1,,14,0,1,0,2,4,happy,Wednesday
37,2024-01-04,3,3,3,3,3,0,3,2,0,...,1,,14,1,2,1,2,4,relaxed,Thursday
38,2024-01-05,3,2,2,3,1,0,3,2,0,...,1,,14,0,3,2,2,3,accomplished,Friday
39,2024-01-06,4,3,3,2,2,0,3,1,0,...,0,,12,0,3,2,2,3,accomplished,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2024-04-15,3,3,3,3,2,3,4,2,0,...,1,1,5,4,2,1,4,3,accomplished,Monday
140,2024-04-16,4,3,4,3,2,0,3,0,1,...,1,1,2,4,1,2,2,3,excited,Tuesday
141,2024-04-17,4,3,4,4,3,4,3,0,0,...,0,1,4,3,2,1,3,3,relaxed,Wednesday
142,2024-04-18,4,4,4,4,4,0,4,1,1,...,1,1,3,2,2,1,3,4,accomplished,Thursday


#### Importing Asleep Data using the Fitbit API

In [9]:
# Importing asleep data gathering script
import gather_asleep_data

start_date = (pd.to_datetime(latest_date) + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
print('Start Date: ',start_date)
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

# Define Fitbit credentials
client_id = ""
client_secret = ""
redirect_uri = '/'

# Instantiate OAuth server
oauth_server = gather_asleep_data.OAuth2Server(client_id, client_secret, redirect_uri)
oauth_server.browser_authorize()

# Access token and refresh token are now available through oauth_server.fitbit.client.session.token
access_token = oauth_server.fitbit.client.session.token['access_token']
refresh_token = oauth_server.fitbit.client.session.token['refresh_token']

# Create an authorized Fitbit API client
fitbit_client = gather_asleep_data.FitbitAPI(client_id, client_secret, access_token, refresh_token, redirect_uri).get_authorized_client()

# Call the function with the start and end dates, and the Fitbit client
raw_sleep_df = gather_asleep_data.fetch_and_format_sleep_data(start_date, end_date, fitbit_client)

[20/Apr/2024:21:46:19] ENGINE Listening for SIGTERM.
[20/Apr/2024:21:46:19] ENGINE Listening for SIGHUP.
[20/Apr/2024:21:46:19] ENGINE Listening for SIGUSR1.
[20/Apr/2024:21:46:19] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[20/Apr/2024:21:46:19] ENGINE Started monitor thread 'Autoreloader'.
[20/Apr/2024:21:46:19] ENGINE Serving on http://127.0.0.1:8080
[20/Apr/2024:21:46:19] ENGINE Bus STARTED


Start Date:  2024-01-01
127.0.0.1 - - [20/Apr/2024:21:46:21] "GET /?code=4f6af6aca8ff0b7fe4207d651e3970048695870a&state=UmpCXVYujKIoVYiobWvumdydiYaCap HTTP/1.1" 200 114 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"


[20/Apr/2024:21:46:22] ENGINE Bus STOPPING
[20/Apr/2024:21:46:22] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[20/Apr/2024:21:46:22] ENGINE Stopped thread 'Autoreloader'.
[20/Apr/2024:21:46:22] ENGINE Bus STOPPED
[20/Apr/2024:21:46:22] ENGINE Bus EXITING
[20/Apr/2024:21:46:22] ENGINE Bus EXITED
[20/Apr/2024:21:46:22] ENGINE Waiting for child threads to terminate...


In [10]:
# Creating a copy to avoid overusing the API
new_asleep_df = raw_sleep_df.copy()
new_asleep_df.rename(columns={'date': 'Date'}, inplace=True)
new_asleep_df['Date'] = pd.to_datetime(new_asleep_df['Date'])

#### Merging the New Data Dataframes and Append to the Database

In [11]:
# Merge new_awake_df and new_asleep_df on their 'Date' columns
merged_df = pd.merge(new_awake_df, new_asleep_df, on='Date', how='outer')

# Merge merged_df with database_df, keeping only new rows
new_data = pd.concat([database_df, merged_df]).drop_duplicates(subset=['Date'], keep='last')

# Drop empty rows
merged_df = merged_df.dropna(how='all')

# Save the updated dataframe to 'database.csv'
new_data.to_csv('database.csv', index=False)